## Notebook 00 – Data Pull

This notebook pulls and exports the data required for this project from EODHD. 

**Note:** To run this notebook, you must set your EODHD API key in `src/config.py`.

Replace the following line in `config.py`:

    API_KEY = ""  # Add your actual key here

Alternatively, you can store the key as an environment variable and update `config.py` like this:

    import os
    API_KEY = os.getenv("EODHD_API_KEY")

### Step 0 - Import packages and functions

In [ ]:
import sys, os
sys.path.append(os.path.abspath("../src"))
sys.path.append("..") 
import pandas as pd
import requests
import time

# Helper functions to pull data
from data_fetcher import fetch_and_save_price

# Import global variables
from config import API_KEY, BASE_URL, DATE_FROM, DATE_TO, OUTPUT_DIR

### Step 1 - Create suffix mapping

In [ ]:
# Create suffix mapping to account for EODHD naming convention
suffix_map = {
    'sp500.csv': '.US',        # US – Covers both NASDAQ and NYSE
}

### Step 2 - Create data folder if needed

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

### Step 3 - Get list of tickers

In [ ]:
dfs = {}
path = '../data'
for file in os.listdir(path):
    if file.endswith('.csv') and file in suffix_map:
        df = pd.read_csv(
            os.path.join(path, file),
            encoding='utf-8-sig',
            dtype={'ticker': str},
            keep_default_na=False
        )
        suffix = suffix_map[file]
        df['ticker'] = df['ticker'].str.strip()

        df['eodhd_ticker'] = df['ticker'].apply(
            lambda x: x if x.endswith(suffix) else x + suffix # change ticker to match EODHD naming conventions
        )
        dfs[file] = df

### Step 4 - Pull and export data

In [ ]:
for fname, df in dfs.items():
    for ticker in df['eodhd_ticker']:
        fetch_and_save_price(ticker, fname, BASE_URL, API_KEY, DATE_FROM, DATE_TO, OUTPUT_DIR)
        time.sleep(1.2)  # Respect rate limits